In [1]:
import math
import wave
import array
import struct
import winsound

import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft

In [2]:
TWO_PI = 2.0 * math.pi
base_path = '../data/kaggle-notes'

sin2pi = lambda x : math.sin(TWO_PI * x)

In [3]:
def play_wav_file(file):
    winsound.PlaySound(file, winsound.SND_ALIAS)

In [4]:
def get_samples(frequencies, samples_per_second, samples_count):
    samples = [ 0 ] * samples_count
    for time in range(samples_count):
        sines = [amplitude * sin2pi(time * freq / samples_per_second) for freq, amplitude in frequencies.items()]
        samples[time] = sum(sines)
        
    return samples

# Frequency: in Hertz
# frequencies: dictionary of frequencies to amplitudes, the latter being in [0, 1]
# Duration: in seconds
# output_file: the file path
def generate_frequencies(frequencies, duration, output_file):
    # Signed short integer (-32768 to 32767) data
    data = array.array('h')
    
    # 2 bytes because of using signed short integers => bit depth = 16
    data_size = 2
    
    number_of_channels = 1
    samples_per_second = 44100
    samples_count = samples_per_second * duration
    
    samples = get_samples(frequencies, samples_per_second, samples_count)
    data = array.array('h', map(lambda x : min(32767, max(-32767, int(32767 * x))), samples))
        
    f = wave.open(output_file, 'w')
    f.setparams((number_of_channels, data_size, samples_per_second, samples_count, "NONE", "Uncompressed"))
    f.writeframes(data.tobytes())
    f.close()

# Shorthand for 1 frequency
def generate_frequency(freq, amplitude, duration, output_file):
    generate_frequencies({freq: amplitude}, duration, output_file)

In [10]:
generate_frequencies({440: 200/32767}, 2, "test2.wav")

In [6]:
def fourier_transform(samples, samples_per_second):
    N = len(samples)
    
    amplitudes = 2.0 / N * np.abs(fft(samples)[0 : N//2])
    frequencies = np.linspace(0.0, samples_per_second/(2), N//2)
    return frequencies, amplitudes

In [7]:
if False:
    samples_count = 200
    samples_per_second = 100
    samples = get_samples({1: 1, 3: 3, 5: -2}, samples_per_second, samples_count)
    frequencies, amplitudes = fourier_transform(samples, samples_per_second)

    plt.plot(frequencies, amplitudes, color='blue')
    plt.xlim(0, 6)
    plt.grid()

    plt.show()

    print("Maximum frequency: ", frequencies[np.argmax(amplitudes)])